In [1]:
import os.path
import mysql.connector
import sqlite3

class DBConnection():
    """DB Connection"""
    def __init__(self, **kwargs):
        self.kwargs = kwargs
        self.connection = None
        
    def __enter__(self):
        self.connection = mysql.connector.connect(**self.kwargs)
        return self.connection
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.connection.close()
                 

class DBManager:
    def __init__(self, mysql, sqlite):
        self.mysql = mysql
        self.sqlite = sqlite
        self._sqlitedb = None
        
    def sqlite_db(self):
        if not self._sqlitedb:
            db = sqlite3.connect(self.sqlite)
            self._sqlitedb = db
        return self._sqlitedb
    
    def mysql_db(self):
        return DBConnection(**self.mysql)
    
    def _init_verbatim(self):
        db = self._sqlitedb
        with db:
            cr = db.cursor()
            q = """
                CREATE TABLE verbatim(
                    id INTEGER PRIMARY KEY,
                    refid TEXT NOT NULL UNIQUE,
                    title TEXT NOT NULL,
                    content TEXT NOT NULL,
                    parent_refid TEXT NOT NULL,
                    parent_type TEXT NOT NULL,
                    type TEXT NOT NULL
                )
            """
            cr.execute(q)
            cr.execute("CREATE INDEX index_verbatim_refid ON verbatim(refid)")
    
    def _init_lead(self):
        db = self._sqlitedb
        with db:
            cr = db.cursor()
            q = """
                CREATE TABLE lead(
                    id INTEGER PRIMARY KEY,
                    refid TEXT NOT NULL UNIQUE,
                    account_refid TEXT,
                    date_entered TEXT NOT NULL,
                    date_modified TEXT NOT NULL,
                    title TEXT,
                    converted BOOLEAN,
                    name TEXT,
                    status TEXT,
                    description TEXT
                )
            """
            cr.execute(q)
            cr.execute("CREATE INDEX index_lead_refid ON lead(refid)")
            cr.execute("CREATE INDEX index_lead_account_refid ON lead(account_refid)")
            
    
    def _init_account(self):
        db = self._sqlitedb
        with db:
            cr = db.cursor()
            q = """
                CREATE TABLE account(
                    id INTEGER PRIMARY KEY,
                    refid TEXT NOT NULL UNIQUE,
                    date_entered TEXT NOT NULL,
                    date_modified TEXT NOT NULL,
                    name TEXT,
                    type TEXT,
                    industry TEXT,
                    description TEXT
                )
            """
            cr.execute(q)
            cr.execute("CREATE INDEX index_account_refid ON account(refid)")
    
    def _init_sqlite(self):
        if os.path.exists(self.sqlite):
            os.remove(self.sqlite)
    
        db = sqlite3.connect(self.sqlite)
        self._sqlitedb = db
        self._init_verbatim()
        self._init_account()
        self._init_lead()
    
    def _insert_verbatims(self, cursor, v_type):
        db = self.sqlite_db()
        insert = "INSERT INTO verbatim VALUES(NULL, ?, ?, ?, ?, ?, ?)"
        with db:
            cr_sqlite = db.cursor()
            for row in cursor:
                r = list(row)[:5] + [v_type]
                cr_sqlite.execute(insert, r)
                    
    def extract_verbatim_meetings(self):
        with self.mysql_db() as mysql_db:
            cr = mysql_db.cursor()
            q = """
                SELECT
                    id,
                    name,
                    description,
                    parent_id,
                    parent_type
                FROM meetings
                WHERE (description is not null AND description != '' AND parent_id is not null AND parent_type is not null)
            """
            cr.execute(q)
            self._insert_verbatims(cr, 'meetings')
                
    def extract_verbatim_calls(self):
        with self.mysql_db() as mysql_db:
            cr = mysql_db.cursor()
            q = """
                SELECT
                    id,
                    name,
                    description,
                    parent_id,
                    parent_type
                FROM calls
                WHERE (description is not null AND description != '' AND parent_id is not null AND parent_type is not null)
            """
            cr.execute(q)
            self._insert_verbatims(cr, 'calls')


    def extract_verbatim_notes(self):
        with self.mysql_db() as mysql_db:
            cr = mysql_db.cursor()
            q = """
                SELECT
                    id,
                    name,
                    description,
                    parent_id,
                    parent_type,
                    filename
                FROM notes
                WHERE (description is not null AND description != '' AND parent_id is not null AND parent_type is not null AND filename is null)
            """
            cr.execute(q)
            self._insert_verbatims(cr, 'notes')
    
    def _extract_verbatims(self):
        self.extract_verbatim_meetings()
        self.extract_verbatim_calls()
        self.extract_verbatim_notes()
        
    def _extract_accounts(self):
        with self.mysql_db() as mysql_db:
            cr = mysql_db.cursor()
            q = """
                SELECT 
                    id,
                    date_entered,
                    date_modified,
                    name,
                    account_type,
                    industry,
                    description
                FROM accounts
            """
            cr.execute(q)
            insert = """INSERT INTO account VALUES (NULL, ?, ?, ?, ?, ?, ?, ?)"""
            db = self.sqlite_db()
            with db:
                cr_sqlite = db.cursor()
                for row in cr:
                    cr_sqlite.execute(insert, row)
 
    def _extract_leads(self):
        with self.mysql_db() as mysql_db:
            cr = mysql_db.cursor()
            q = """
                SELECT 
                    id,
                    account_id,
                    date_entered,
                    date_modified,
                    title,
                    converted,
                    first_name || last_name,
                    status,
                    description
                FROM leads
            """
            cr.execute(q)
            insert = """INSERT INTO lead VALUES (NULL, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""
            db = self.sqlite_db()
            with db:
                cr_sqlite = db.cursor()
                for row in cr:
                    cr_sqlite.execute(insert, row)

In [2]:
mysqlconf = {
  'user': 'chaimaa',
  'password': 'chaimaa',
  'host': 'localhost',
  'database': 'verbatim'
}
dbmanager = DBManager(mysqlconf, "../data/datasets/verbatim.sqlite")

In [4]:
dbmanager._init_sqlite()
dbmanager._extract_accounts()
dbmanager._extract_leads()
dbmanager._extract_verbatims()

In [ ]:
dbmanager._extract_leads()
